## IMU Low Back Data Processing

Specs:
- IMeasureU Blue Trident Senors
- Sampling freq for lowg = 1125hz
- Sampling freq for highg = 1600hz
- 5 minutes of data = 337,500 rows (1125hz) or 478,000 (1600hz)

In [ ]:
# Import custom functions ---
import functions.file_import_gui as gui
import functions.data_prep as prep
import functions.custom_plots as plots
import functions.low_back_measures as back
import functions.peak_detection as peaks
import functions.stats as stats

# For saving files
import os

# For dataframes ---
import pandas as pd

# For plotting ---
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Bring in IMU data files ---

# Subject to process
sub_id = 'run014'

# lowg ---
# set directory
initialdir = f"data/five_min_runs/{sub_id}/lowg_1125hz/back"
# bring in csv files with data
dfs_lowg, keys_list = gui.read_csv_files_gui(initialdir)

# highg ---
# set directory
initialdir = f"data/five_min_runs/{sub_id}/highg_1600hz/back"
# bring in csv files with data
dfs_highg, keys_list = gui.read_csv_files_gui(initialdir)

Prep

In [ ]:
# Data prep ---

# lowg ---
# crop data for 5 mins (removes extra rows at the beginning so that there are exactly 5 mins of data)
prep.crop_df_five_mins(dfs_lowg, sample_freq = 1125)
# calculate and add resultant column
prep.add_resultant_column(dfs_lowg, column_x = 'ax_m/s/s', column_y = 'ay_m/s/s', column_z = 'az_m/s/s', name_of_res_column = 'res_m/s/s')
# convert accel columns to gs
prep.accel_to_gs_columns(dfs_lowg)
# shift time scale to start at 0
prep.shift_time_s_to_zero(dfs_lowg)


# highg ---
# crop data for 5 mins (removes extra rows at the beginning so that there are exactly 5 mins of data)
prep.crop_df_five_mins(dfs_highg, sample_freq = 1600)
# calculate and add resultant column
prep.add_resultant_column(dfs_highg, column_x = 'ax_m/s/s', column_y = 'ay_m/s/s', column_z = 'az_m/s/s', name_of_res_column = 'res_m/s/s')
# convert accel columns to gs
prep.accel_to_gs_columns(dfs_highg)
# shift time scale to start at 0
prep.shift_time_s_to_zero(dfs_highg)

In [ ]:
# Mean-Shift Values

# lowg ---
# meanshift
prep.calc_mean_shift(dfs_lowg, ['ax_m/s/s', 'ay_m/s/s', 'az_m/s/s', 'ax_g', 'ay_g', 'az_g'])
# calculate resultant from meanshifted signals
prep.add_resultant_column(
    dfs_lowg, column_x = 'ax_m/s/s_meanshift', column_y = 'ay_m/s/s_meanshift', column_z = 'az_m/s/s_meanshift', name_of_res_column = 'res_m/s/s_meanshift'
    )
prep.add_resultant_column(
    dfs_lowg, column_x = 'ax_g_meanshift', column_y = 'ay_g_meanshift', column_z = 'az_g_meanshift', name_of_res_column = 'res_g_meanshift'
    )

# highg ---
# meanshift
prep.calc_mean_shift(dfs_highg, ['ax_m/s/s', 'ay_m/s/s', 'az_m/s/s', 'ax_g', 'ay_g', 'az_g'])
# calculate resultant from meanshifted signals
prep.add_resultant_column(
    dfs_highg, column_x = 'ax_m/s/s_meanshift', column_y = 'ay_m/s/s_meanshift', column_z = 'az_m/s/s_meanshift', name_of_res_column = 'res_m/s/s_meanshift'
    )
prep.add_resultant_column(
    dfs_highg, column_x = 'ax_g_meanshift', column_y = 'ay_g_meanshift', column_z = 'az_g_meanshift', name_of_res_column = 'res_g_meanshift'
    )

In [ ]:
# Filter data ---

cutoff_frequency = 50 #hz
filter_order = 4 #th

# lowg ---
sampling_frequency = 1125 #hz 
# Filter raw data
columns_to_filter = [
    'ax_m/s/s', 'ay_m/s/s', 'az_m/s/s', 'res_m/s/s', 
    'ax_g', 'ay_g', 'az_g', 'res_g']
prep.apply_butter_lowpass_filter_to_dfs(dfs_lowg, columns_to_filter, sampling_frequency, cutoff_frequency, filter_order)
# Filter mean shift data
columns_to_filter = [
    'ax_m/s/s_meanshift', 'ay_m/s/s_meanshift', 'az_m/s/s_meanshift', 'res_m/s/s_meanshift', 
    'ax_g_meanshift', 'ay_g_meanshift', 'az_g_meanshift', 'res_g_meanshift']
prep.apply_butter_lowpass_filter_to_dfs(dfs_lowg, columns_to_filter, sampling_frequency, cutoff_frequency, filter_order)

# highg ---
sampling_frequency = 1600 #hz 
# Filter raw data
columns_to_filter = [
    'ax_m/s/s', 'ay_m/s/s', 'az_m/s/s', 'res_m/s/s', 
    'ax_g', 'ay_g', 'az_g', 'res_g']
prep.apply_butter_lowpass_filter_to_dfs(dfs_highg, columns_to_filter, sampling_frequency, cutoff_frequency, filter_order)
# Filter mean shift data
columns_to_filter = [
    'ax_m/s/s_meanshift', 'ay_m/s/s_meanshift', 'az_m/s/s_meanshift', 'res_m/s/s_meanshift', 
    'ax_g_meanshift', 'ay_g_meanshift', 'az_g_meanshift', 'res_g_meanshift']
prep.apply_butter_lowpass_filter_to_dfs(dfs_highg, columns_to_filter, sampling_frequency, cutoff_frequency, filter_order)

Create plots to save in folder

In [ ]:
# Create and store plots for specified columns ---

x_col = 'time_s_scaled'
y_cols = ['ax_m/s/s_meanshift_filtered', 'ay_m/s/s_meanshift_filtered', 'az_m/s/s_meanshift_filtered', 'res_m/s/s_meanshift_filtered']

# lowg ---
line_plots_lowg = plots.create_line_plots(dfs_lowg, x_col, y_cols)
# highg ---
line_plots_highg = plots.create_line_plots(dfs_highg, x_col, y_cols)

In [ ]:
# Save plots to folder ---

# lowg --
# set directory
output_dir = f"plots/{sub_id}/back/lowg_1125hz/"
# check if directory exists, if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# loop through all plots and save them
for key, fig in line_plots_lowg.items():
    file_path = os.path.join(output_dir, f"{key}.html")
    pio.write_html(fig, file_path)

# highg --
# set directory
output_dir = f"plots/{sub_id}/back/highg_1600hz/"
# check if directory exists, if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# loop through all plots and save them
for key, fig in line_plots_highg.items():
    file_path = os.path.join(output_dir, f"{key}.html")
    pio.write_html(fig, file_path)

### Center of Mass (CoM) Measures
- **Root Mean Squared (RMS)** - a single value for each axis of acceleration (m/s/s) VT, ML, AP, resultant (RES)
- **RMS Ratio** - the RMS of each axis is divided by the resultant root mean squared, for example VT_RMS/RES_RMS
- **AVG Peak Acceleration of Resultant** - finds peaks of resultant and calculates the avg of these peaks

**NOTE:** 
- The RMS and RMS Ratio are *mean-shifted* and calculated from the *filtered* signal
- AVG Peak Accel uses the *raw* signal (no mean-shift or filtering)

Axis Orientation:
- **X-axis**: represents the **medial-lateral (ML)** direction, with positive values pointing to the right and negative values pointing to the left. This corresponds to the side-to-side movement of the body.
- **Y-axis**: aligned with the **vertical (VT)** direction, with positive values indicating a superior (upward) direction and negative values indicating an inferior (downward) direction. This corresponds to the up and down movement of the body.
- **Z-axis**: oriented in the **anterior-posterior (AP)** direction, with positive values pointing anterior (forward) and negative values pointing posterior (backward). This represents the forward and backward movement of the body.

#### RMS

In [ ]:
# RMS Calculations ---

columns_for_rms = [
    'ax_m/s/s', 'ay_m/s/s', 'az_m/s/s', 'res_m/s/s', # raw
    'ax_m/s/s_filtered', 'ay_m/s/s_filtered', 'az_m/s/s_filtered', 'res_m/s/s_filtered', # filtered
    'ax_m/s/s_meanshift_filtered', 'ay_m/s/s_meanshift_filtered', 'az_m/s/s_meanshift_filtered', 'res_m/s/s_meanshift_filtered' # mean shifted & filtered
    ]
# Use custom function (back.apply_rms_to_dfs)
# returns a table in long format (variables are in a column and the values are in another)
# also adds suffix at end of each value in the column variable (need this later to know which hz)
rms_df_lowg = back.apply_rms_to_dfs(dfs_lowg, columns_for_rms)
rms_df_lowg['variable'] = rms_df_lowg['variable'] + '_1125hz'
rms_df_highg = back.apply_rms_to_dfs(dfs_highg, columns_for_rms)
rms_df_highg['variable'] = rms_df_highg['variable'] + '_1600hz'


Visually Inspect Distribution of Variable

In [ ]:
# lowg ---

variables = rms_df_lowg["variable"].unique()  # assuming variable is categorical

fig = make_subplots(rows=3, cols=4)  # create 3x4 grid of subplots

for i, var in enumerate(variables):
    row = i // 4 + 1  # calculate the row of the subplot
    col = i % 4 + 1   # calculate the column of the subplot
    box_data = rms_df_lowg[rms_df_lowg["variable"] == var]["value"]
    fig.add_trace(go.Box(y=box_data, quartilemethod='exclusive', name=var), row=row, col=col)

fig.update_xaxes(title='', tickangle=0, side='top')  # moves x-axis labels to the top
fig.update_yaxes(title='')
fig.update_layout(title_text='Low G', height=800, showlegend=False)

fig.show()

# highg ---

variables = rms_df_highg["variable"].unique()  # assuming variable is categorical

fig = make_subplots(rows=3, cols=4)  # create 3x4 grid of subplots

for i, var in enumerate(variables):
    row = i // 4 + 1  # calculate the row of the subplot
    col = i % 4 + 1   # calculate the column of the subplot
    box_data = rms_df_highg[rms_df_highg["variable"] == var]["value"]
    fig.add_trace(go.Box(y=box_data, quartilemethod='exclusive', name=var), row=row, col=col)

fig.update_xaxes(title='', tickangle=0, side='top')  # moves x-axis labels to the top
fig.update_yaxes(title='')
fig.update_layout(title_text='High G', height=800, showlegend=False)

fig.show()

In [ ]:
# Create table to export & append rows to to my processed variables table in Excel ---

# lowg ---
df_to_export = prep.export_tbl(rms_df_lowg)
file_path = "data/processed_variables/imu_training_load_variables.xlsx"
sheet_name = "variables"
prep.append_df_to_excel(df_to_export, file_path, sheet_name)

# highg ---
df_to_export = prep.export_tbl(rms_df_highg)
file_path = "data/processed_variables/imu_training_load_variables.xlsx"
sheet_name = "variables"
prep.append_df_to_excel(df_to_export, file_path, sheet_name)

#### RMS Ratios

In [ ]:
# RMS Ratio Calculations ---

# lowg ---

# filtered signal ---
# pivot the rms df so its easier to work with
pivot_rms_df_lowg_wide = rms_df_lowg.pivot(index='key', columns='variable', values='value')
# calculate the ratios for each column (variable)
for axis in ['ax_m/s/s_filtered_rms_1125hz', 'ay_m/s/s_filtered_rms_1125hz', 'az_m/s/s_filtered_rms_1125hz']:
    pivot_rms_df_lowg_wide[axis+'_ratio'] = pivot_rms_df_lowg_wide[axis] / pivot_rms_df_lowg_wide['res_m/s/s_filtered_rms_1125hz']
# keep only the ratio columns
pivot_rms_df_lowg_wide = pivot_rms_df_lowg_wide[[col for col in pivot_rms_df_lowg_wide.columns if 'ratio' in col]]
# melt the df back to a long format
rms_filtered_ratio_df_lowg = pivot_rms_df_lowg_wide.reset_index().melt(id_vars='key', var_name='variable', value_name='value')

# mean shifted filtered signal ---
# pivot the rms df so its easier to work with
pivot_rms_df_lowg_wide = rms_df_lowg.pivot(index='key', columns='variable', values='value')
# calculate the ratios for each column (variable)
for axis in ['ax_m/s/s_meanshift_filtered_rms_1125hz', 'ay_m/s/s_meanshift_filtered_rms_1125hz', 'az_m/s/s_meanshift_filtered_rms_1125hz']:
    pivot_rms_df_lowg_wide[axis+'_ratio'] = pivot_rms_df_lowg_wide[axis] / pivot_rms_df_lowg_wide['res_m/s/s_meanshift_filtered_rms_1125hz']
# keep only the ratio columns
pivot_rms_df_lowg_wide = pivot_rms_df_lowg_wide[[col for col in pivot_rms_df_lowg_wide.columns if 'ratio' in col]]
# melt the df back to a long format
rms_meanshift_filtered_ratio_df_lowg = pivot_rms_df_lowg_wide.reset_index().melt(id_vars='key', var_name='variable', value_name='value')

# combine tables above
rms_ratio_df_lowg = pd.concat([rms_filtered_ratio_df_lowg, rms_meanshift_filtered_ratio_df_lowg], axis=0, ignore_index=True)

# highg ---

# filtered signal ---
# pivot the rms df so its easier to work with
pivot_rms_df_highg_wide = rms_df_highg.pivot(index='key', columns='variable', values='value')
# calculate the ratios for each column (variable)
for axis in ['ax_m/s/s_filtered_rms_1600hz', 'ay_m/s/s_filtered_rms_1600hz', 'az_m/s/s_filtered_rms_1600hz']:
    pivot_rms_df_highg_wide[axis+'_ratio'] = pivot_rms_df_highg_wide[axis] / pivot_rms_df_highg_wide['res_m/s/s_filtered_rms_1600hz']
# keep only the ratio columns
pivot_rms_df_highg_wide = pivot_rms_df_highg_wide[[col for col in pivot_rms_df_highg_wide.columns if 'ratio' in col]]
# melt the df back to a long format
rms_filtered_ratio_df_highg = pivot_rms_df_highg_wide.reset_index().melt(id_vars='key', var_name='variable', value_name='value')

# mean shifted filtered signal ---
# pivot the rms df so its easier to work with
pivot_rms_df_highg_wide = rms_df_highg.pivot(index='key', columns='variable', values='value')
# calculate the ratios for each column (variable)
for axis in ['ax_m/s/s_meanshift_filtered_rms_1600hz', 'ay_m/s/s_meanshift_filtered_rms_1600hz', 'az_m/s/s_meanshift_filtered_rms_1600hz']:
    pivot_rms_df_highg_wide[axis+'_ratio'] = pivot_rms_df_highg_wide[axis] / pivot_rms_df_highg_wide['res_m/s/s_meanshift_filtered_rms_1600hz']
# keep only the ratio columns
pivot_rms_df_highg_wide = pivot_rms_df_highg_wide[[col for col in pivot_rms_df_highg_wide.columns if 'ratio' in col]]
# melt the df back to a long format
rms_meanshift_filtered_ratio_df_highg = pivot_rms_df_highg_wide.reset_index().melt(id_vars='key', var_name='variable', value_name='value')

# combine tables above
rms_ratio_df_highg = pd.concat([rms_filtered_ratio_df_highg, rms_meanshift_filtered_ratio_df_highg], axis=0, ignore_index=True)

Visually Inspect Distribution of Variable

In [ ]:
# lowg ---

variables = rms_ratio_df_lowg["variable"].unique()  # assuming variable is categorical

fig = make_subplots(rows=2, cols=3)  # create 2x3 grid of subplots

for i, var in enumerate(variables):
    row = i // 3 + 1  # calculate the row of the subplot
    col = i % 3 + 1   # calculate the column of the subplot
    box_data = rms_ratio_df_lowg[rms_ratio_df_lowg["variable"] == var]["value"]
    fig.add_trace(go.Box(y=box_data, quartilemethod='exclusive', name=var), row=row, col=col)

fig.update_xaxes(title='', tickangle=0, side='top')  # moves x-axis labels to the top
fig.update_yaxes(title='')
fig.update_layout(title_text='Low G', height=600, showlegend=False)

fig.show()

# highg ---
variables = rms_ratio_df_highg["variable"].unique()  # assuming variable is categorical

fig = make_subplots(rows=2, cols=3)  # create 2x3 grid of subplots

for i, var in enumerate(variables):
    row = i // 3 + 1  # calculate the row of the subplot
    col = i % 3 + 1   # calculate the column of the subplot
    box_data = rms_ratio_df_highg[rms_ratio_df_highg["variable"] == var]["value"]
    fig.add_trace(go.Box(y=box_data, quartilemethod='exclusive', name=var), row=row, col=col)

fig.update_xaxes(title='', tickangle=0, side='top')  # moves x-axis labels to the top
fig.update_yaxes(title='')
fig.update_layout(title_text='High G', height=600, showlegend=False)

fig.show()

In [ ]:
# Create table to export & append rows to to my processed variables table in Excel ---

# lowg --
df_to_export = prep.export_tbl(rms_ratio_df_lowg)
file_path = "data/processed_variables/imu_training_load_variables.xlsx"
sheet_name = "variables"
prep.append_df_to_excel(df_to_export, file_path, sheet_name)

# highg --
df_to_export = prep.export_tbl(rms_ratio_df_highg)
file_path = "data/processed_variables/imu_training_load_variables.xlsx"
sheet_name = "variables"
prep.append_df_to_excel(df_to_export, file_path, sheet_name)

#### AVG Peak Acceleration of Resultant

In [ ]:
# Set parameters for functions below
# The minimum time between peaks i.e. footstrikes corresponding to 0.25 secs (left *and* right)
lowg_time_between_peaks = 281
highg_time_between_peaks = 400

In [ ]:
# STEP 1:

# Find peaks with no thresholds
# NOTE: This will be used to then determine individual thresholds for max peak height

# lowg ---
_, dfs_res_peak_values_lowg_no_threshold = peaks.calc_avg_positive_peaks(
    dfs_lowg, columns=['res_g'], time_column='time_s_scaled', 
    min_peak_height=None, max_peak_height=None,
    min_samples_between_peaks=lowg_time_between_peaks
)

# highg ---
_, dfs_res_peak_values_highg_no_threshold = peaks.calc_avg_positive_peaks(
    dfs_highg, columns=['res_g'], time_column='time_s_scaled', 
    min_peak_height=None, max_peak_height=None,
    min_samples_between_peaks=highg_time_between_peaks
)

In [ ]:
# STEP 2: 

# Determine subject's individual thresholds for max_peak_height and min_peak_height from peaks indentified w/ no threshold

k = 4 #IQR 
z = 4 #SDs

# lowg ---
# summary table w/upper bound
summary_tbl_lowg = stats.create_summary_tbl(dfs_res_peak_values_lowg_no_threshold, ['peak_values'], k=k, z=z)

# highg ---
# summary table w/upper bound
summary_tbl_highg = stats.create_summary_tbl(dfs_res_peak_values_highg_no_threshold, ['peak_values'], k=k, z=z)

In [ ]:
# STEP 3:

# Use individualized max and min peak height threshold as upper limit for finding peaks
# NOTE: This uses a different peak function that takes the summary table as inputs and steps through the indiv rows of each run/sensor

# lowg ---
res_peak_accel_lowg_df, dfs_res_peak_values_lowg_threshold = peaks.calc_avg_positive_peaks_from_tbl(
    dfs_lowg, ['res_g'], time_column='time_s_scaled',
    summary_table=summary_tbl_lowg, id_column="id", min_peak_height_column="lower_bound_k", max_peak_height_column="upper_bound_k",
    min_samples_between_peaks=lowg_time_between_peaks
    )

# highg ---
res_peak_accel_highg_df, dfs_res_peak_values_highg_threshold = peaks.calc_avg_positive_peaks_from_tbl(
    dfs_highg, ['res_g'], time_column='time_s_scaled',
    summary_table=summary_tbl_highg, id_column="id", min_peak_height_column="lower_bound_k", max_peak_height_column="upper_bound_k",
    min_samples_between_peaks=highg_time_between_peaks
    )

Visually Inspect Peaks

In [ ]:
# Set plot keys for specific runs to visualize below ---

# lowg ---
plot_keys_lowg = [
    f'{sub_id}_heavyd1_prs_post_01010_lowg', 
    f'{sub_id}_heavy_prs_pre_01010_lowg', 
]

# highg ---
plot_keys_highg = [
    f'{sub_id}_heavyd1_prs_post_01010_highg', 
    f'{sub_id}_heavy_prs_pre_01010_highg', 
]

# Create dictionary with dfs I want to plot

# lowg ---
dfs_lowg_plots = {key: dfs_lowg[key] for key in plot_keys_lowg if key in dfs_lowg}

# highg ---
dfs_highg_plots = {key: dfs_highg[key] for key in plot_keys_highg if key in dfs_highg}

# Create and store plots for specified columns ---

x_col = 'time_s_scaled'
y_cols = ['res_g']

# lowg ---
line_plots_lowg = plots.create_line_plots(dfs_lowg_plots, x_col, y_cols)

# highg ---
line_plots_highg = plots.create_line_plots(dfs_highg_plots, x_col, y_cols)

In [ ]:
# lowg ---

peaks_to_plot = dfs_res_peak_values_lowg_threshold

for key in plot_keys_lowg:
    fig = line_plots_lowg.get(key)
    if fig is not None:
        # If 'peaks_2' trace already exists, remove it before adding new one
        fig.data = [trace for trace in fig.data if trace.name != 'peaks']

        # Check if key exists in peaks_to_plot
        if key in peaks_to_plot:
            # Get corresponding DataFrame
            df_lowg_peaks = peaks_to_plot[key]
            # Add points to figure
            fig.add_trace(go.Scatter(
                x=df_lowg_peaks['time_s_scaled'], 
                y=df_lowg_peaks['peak_values'], 
                mode='markers',
                marker=dict(
                    size=8,
                    color='black',  # for example, choose a color that stands out
                ),
                name='peaks'  # you can name the trace to be referenced in legend
            ))
        fig.show()
    else:
        print(f"No plot found with key {key}")

In [ ]:
# highg ---

peaks_to_plot = dfs_res_peak_values_highg_threshold

for key in plot_keys_highg:
    fig = line_plots_highg.get(key)
    if fig is not None:
        # If 'peaks_2' trace already exists, remove it before adding new one
        fig.data = [trace for trace in fig.data if trace.name != 'peaks']

        # Check if key exists in peaks_to_plot
        if key in peaks_to_plot:
            # Get corresponding DataFrame
            df_highg_peaks = peaks_to_plot[key]
            # Add points to figure
            fig.add_trace(go.Scatter(
                x=df_highg_peaks['time_s_scaled'], 
                y=df_highg_peaks['peak_values'], 
                mode='markers',
                marker=dict(
                    size=8,
                    color='black',  # for example, choose a color that stands out
                ),
                name='peaks'  # you can name the trace to be referenced in legend
            ))
        fig.show()
    else:
        print(f"No plot found with key {key}")

Export Variables to Excel Table

In [ ]:
# Add suffixes to variables

# lowg ---
res_peak_accel_lowg_df['variable'] = res_peak_accel_lowg_df['variable'] + '_back_1125hz'
# highg ---
res_peak_accel_highg_df['variable'] = res_peak_accel_highg_df['variable'] + '_back_1600hz'

In [ ]:
# Create table to export & append rows to to my processed variables table in Excel ---

# lowg --
df_to_export = prep.export_tbl(res_peak_accel_lowg_df)
file_path = "data/processed_variables/imu_training_load_variables.xlsx"
sheet_name = "variables"
prep.append_df_to_excel(df_to_export, file_path, sheet_name)

# highg --
df_to_export = prep.export_tbl(res_peak_accel_highg_df)
file_path = "data/processed_variables/imu_training_load_variables.xlsx"
sheet_name = "variables"
prep.append_df_to_excel(df_to_export, file_path, sheet_name)
